# Bidirectional RNN (BiRNN) - our task is to implement and research about BiRNN.

In [1]:
# Apply RNN basic 
# We'll create a simple model that learns to predict the next character in a sequence of characters from a given text. 

In [22]:
import torch
import torch.nn as nn
import numpy as np
import random
import string

In [23]:
# Define a simple text corpus
text = "Neural networks, or artificial neural networks (ANNs), are comprised of node layers, containing an input layer, one or more hidden layers, and an output layer. Each node, or artificial neuron, connects to another and has an associated weight and threshold. If the output of any individual node is above the specified threshold value, that node is activated, sending data to the next layer of the network. Otherwise, no data is passed along to the next layer of the network by that node. The “deep” in deep learning is just referring to the number of layers in a neural network. A neural network that consists of more than three layers—which would be inclusive of the input and the output—can be considered a deep learning algorithm or a deep neural network. A neural network that only has three layers is just a basic neural network.."
text += " " + string.ascii_lowercase  # Adding the alphabet to our corpus for diversity

# Create a mapping of characters to integers
chars = sorted(set(text))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}


char_to_int

{' ': 0,
 '(': 1,
 ')': 2,
 ',': 3,
 '.': 4,
 'A': 5,
 'E': 6,
 'I': 7,
 'N': 8,
 'O': 9,
 'T': 10,
 'a': 11,
 'b': 12,
 'c': 13,
 'd': 14,
 'e': 15,
 'f': 16,
 'g': 17,
 'h': 18,
 'i': 19,
 'j': 20,
 'k': 21,
 'l': 22,
 'm': 23,
 'n': 24,
 'o': 25,
 'p': 26,
 'q': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'x': 34,
 'y': 35,
 'z': 36,
 '—': 37,
 '“': 38,
 '”': 39}

In [24]:
# Prepare dataset
def char_sequences(text, seq_length):
    inputs = []
    targets = []
    for i in range(len(text) - seq_length):
        input_seq = text[i:i+seq_length]
        target = text[i+seq_length]
        inputs.append([char_to_int[char] for char in input_seq])
        targets.append(char_to_int[target])
    return np.array(inputs), np.array(targets)

seq_length = 10  # Length of the sequence to be taken as input

X, y = char_sequences(text, seq_length)


In [26]:
# Convert to tensors
X = torch.from_numpy(X).to(torch.int64)
y = torch.from_numpy(y).to(torch.int64)

In [32]:
y

tensor([33, 25, 28, 21, 29,  3,  0, 25, 28,  0, 11, 28, 30, 19, 16, 19, 13, 19,
        11, 22,  0, 24, 15, 31, 28, 11, 22,  0, 24, 15, 30, 33, 25, 28, 21, 29,
         0,  1,  5,  8,  8, 29,  2,  3,  0, 11, 28, 15,  0, 13, 25, 23, 26, 28,
        19, 29, 15, 14,  0, 25, 16,  0, 24, 25, 14, 15,  0, 22, 11, 35, 15, 28,
        29,  3,  0, 13, 25, 24, 30, 11, 19, 24, 19, 24, 17,  0, 11, 24,  0, 19,
        24, 26, 31, 30,  0, 22, 11, 35, 15, 28,  3,  0, 25, 24, 15,  0, 25, 28,
         0, 23, 25, 28, 15,  0, 18, 19, 14, 14, 15, 24,  0, 22, 11, 35, 15, 28,
        29,  3,  0, 11, 24, 14,  0, 11, 24,  0, 25, 31, 30, 26, 31, 30,  0, 22,
        11, 35, 15, 28,  4,  0,  6, 11, 13, 18,  0, 24, 25, 14, 15,  3,  0, 25,
        28,  0, 11, 28, 30, 19, 16, 19, 13, 19, 11, 22,  0, 24, 15, 31, 28, 25,
        24,  3,  0, 13, 25, 24, 24, 15, 13, 30, 29,  0, 30, 25,  0, 11, 24, 25,
        30, 18, 15, 28,  0, 11, 24, 14,  0, 18, 11, 29,  0, 11, 24,  0, 11, 29,
        29, 25, 13, 19, 11, 30, 15, 14, 

In [28]:
class TextGeneratorRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TextGeneratorRNN, self).__init__()
        self.embed = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out[:, -1, :])  # Get the output for the last time step
        return out, hidden

input_size = len(chars)
hidden_size = 100
output_size = len(chars)

model = TextGeneratorRNN(input_size, hidden_size, output_size)


In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output, _ = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 20 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 20, Loss: 1.5448832511901855
Epoch 40, Loss: 0.7528386116027832
Epoch 60, Loss: 0.2651006877422333
Epoch 80, Loss: 0.08779888600111008
Epoch 100, Loss: 0.05244646593928337
Epoch 120, Loss: 0.043429240584373474
Epoch 140, Loss: 0.039631426334381104
Epoch 160, Loss: 0.03759302943944931
Epoch 180, Loss: 0.036369722336530685
Epoch 200, Loss: 0.03540007397532463


In [30]:
def generate_text(model, start_str='uk weather', length=100):
    model.eval()
    start_input = torch.tensor([char_to_int[c] for c in start_str], dtype=torch.int64).unsqueeze(0)
    generated_text = start_str

    hidden = None
    for _ in range(length):
        output, hidden = model(start_input, hidden)
        probabilities = torch.softmax(output, dim=1).detach().numpy().flatten()
        next_char_id = np.random.choice(len(chars), p=probabilities)
        next_char = int_to_char[next_char_id]
        generated_text += next_char

        start_input = torch.tensor([[next_char_id]], dtype=torch.int64)

    return generated_text

# Generate and print some text
print(generate_text(model, start_str="talk about uk", length=500))


talk about uks passed algorithm or ar is ano neural network.. abcinclaye, leat lanetwork. A neural network. A neural network. A neural network. A neural network. Otherwise, no data twork by that consists of more hijuac network. Otherwise, no the output of the network that cone connects is just a basic neural network. A neural network that cone consed of noded has an as an artividual node is aconon, or artificial networks (ANNs), are or artificial network.. abconsider an noNgoring node, of the nutumprre, that
